# My google places class

In [1]:
from urllib.parse import urlencode,urlparse, parse_qsl
import os,requests
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_KEY")


### Defining all the functions implemented in the google_API.ipynb into a methods of a new class: Places.


In [210]:
class Places (object):
    lat = None
    lng = None
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_zip = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API key is required")
        self.api_key = api_key
        self.location_query = address_zip
        if self.location_query != None:
            self.extract_lat_lng()
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpoint = "https://maps.googleapis.com/maps/api/geocode/json"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299): 
            return {}
        latlng = {}
        try:
            latlng = r.json()['results'][0]['geometry']['location']
        except:
            pass
        lat,lng = latlng.get("lat"), latlng.get("lng")
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword="Starbucks", radius=1000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location=location)
        endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": radius,
            "keyword": keyword
        }
        params_encoded = urlencode(params)
        places_url = f"{endpoint}?{params_encoded}"
        r = requests.get(places_url)
        return r.json()

# Function to extract the desire requirement

In [2]:
def find(query, token=None, maximum = 50):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "key": GOOGLE_API_KEY,
        "query": query
        }
    if token:
        params["pagetoken"] = token
    res = requests.get(url, params=params).json()
    results = res["results"]
    if 'next_page_token' in res.keys():
        token = res['next_page_token']
        r=[results, token]
    else: 
        r=[results,False]
    ubis = []
    token = None
    
    while len(ubis) < maximum:
        results, token = r
        ubis += results
        if not token: 
            break
    return ubis

# Function to extract the location:

In [3]:
def search(place,quantity):
    r = find(query=place, maximum = quantity)
    ad=[]
    latlng=[]
    for i in range(len(r)):
        ad.append(r[i]['formatted_address'])
        lat = r[i]['geometry']['location'].get("lat")
        lng = r[i]['geometry']['location'].get("lat")
        latlng.append([lat,lng])
    return ad,latlng

In [4]:
search(place = "Starbucks in California", quantity = 100)

(['3595 California St, San Francisco, CA 94118, United States',
  '580 California Street, California St #150, San Francisco, CA 94104, United States',
  '52 California St, San Francisco, CA 94111, United States',
  '3700 California Ave Suite #400, Bakersfield, CA 93309, United States',
  '11245 California St #A, Loma Linda, CA 92354, United States',
  '1200 Chester Ave, Bakersfield, CA 93301, United States',
  '1545 California Cir, Milpitas, CA 95035, United States',
  '361 California Ave, Palo Alto, CA 94306, United States',
  '295 California St, San Francisco, CA 94111, United States',
  '5171 California Ave #120, Irvine, CA 92617, United States',
  '1269 California Ave, Pittsburg, CA 94565, United States',
  '41032 California Oaks Rd, Murrieta, CA 92562, United States',
  '2435 California St, San Francisco, CA 94115, United States',
  '1687 N California St #1, Stockton, CA 95204, United States',
  '155 W California Blvd, Pasadena, CA 91105, United States',
  '555 California St, San 